In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('data.csv')
df2 = pd.read_csv('data_2genre.csv')
df = pd.concat([df1, df2])

In [3]:
genres = []

for _ in df['filename']:
    genres.append(_[:-9])

df.insert(1, 'genre', genres)

In [4]:
# bins = [45, 66, 76, 108, 120,
#        156, 176, 200, 235
# ]
# labels = ['Largo', 'Adagio', 'Andante',
#          'Moderato', 'Allegro', 'Vivace',
#          'Presto', 'Prestissimo'
# ]
# df['Tempo Markings'] = pd.cut(df['tempo'], bins=bins, labels=labels)

In [4]:
df = df[['genre', 'beats', 'chroma_stft', 'rmse',
       'spectral_centroid', 'spectral_bandwidth', 'rolloff',
       'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5',
       'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12',
       'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19',
       'mfcc20']]
df.head()

,genre,beats,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,blues,50,0.380260,0.248262,2116.942959,1956.611056,4196.107960,0.127272,-26.929785,107.334008,...,-15.234050,14.336612,-13.821769,7.562789,-6.181372,0.330165,-6.829571,0.965922,-7.570825,2.918987
1,blues,44,0.306451,0.113475,1156.070496,1497.668176,2170.053545,0.058613,-233.860772,136.170239,...,5.145258,-2.250578,3.959198,5.322555,0.812028,-1.107202,-4.556555,-2.436490,3.316913,-0.608485
2,blues,75,0.253487,0.151571,1331.073970,1973.643437,2900.174130,0.042967,-221.802549,110.843070,...,-26.406674,-13.037723,-12.652228,-1.821905,-7.260097,-6.660252,-14.682694,-11.719264,-11.025216,-13.387260
3,blues,91,0.269320,0.119072,1361.045467,1567.804596,2739.625101,0.069124,-207.208080,132.799175,...,-2.452068,-0.613248,0.384877,2.605128,-5.188924,-9.527455,-9.244394,-2.848274,-1.418707,-5.932607
4,blues,74,0.391059,0.137728,1811.076084,2052.332563,3927.809582,0.075480,-145.434568,102.829023,...,-6.934599,7.457218,-10.470444,-2.360483,-6.783623,2.671134,-4.760879,-0.949005,0.024832,-2.005315


In [7]:
!pip install pandas-profiling

In [26]:
df['genre'].unique()

array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
       'metal', 'pop', 'reggae', 'rock'], dtype=object)

In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, train_size=0.8, test_size=0.2, random_state=42)

In [6]:
train, val = train_test_split(train, train_size=0.8, test_size=0.2, random_state=42)

In [7]:
!pip install category-encoders

In [7]:
import category_encoders as ce

In [8]:
def wrangle(train, val, test):
    
    X_train = train.drop(columns='genre')
    y_train = train['genre']
    X_val = val.drop(columns='genre')
    y_val = val['genre']
    X_test = test.drop(columns='genre')
    
    encoder = ce.OrdinalEncoder()

    y_train = encoder.fit_transform(y_train)
    y_val = encoder.transform(y_val)
    
    return X_train, y_train, X_val, y_val, X_test

In [9]:
X_train, y_train, X_val, y_val, X_test = wrangle(train, val, test)

In [10]:
for df in X_train, y_train, X_val, y_val, X_test:
    print(df.shape)

(768, 27)
(768, 1)
(192, 27)
(192, 1)
(240, 27)


In [18]:
import numpy as np
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', multi_class='ovr')

model.fit(X_train, np.ravel(y_train))
print('Validation Accuracy', model.score(X_val, np.ravel(y_val)))

Validation Accuracy 0.640625
